In [ ]:
"""
Tensork network implementation of AVQITE calculation of matrix M_{\mu\nu}.
This implementation utilizes both Quimb for TN contraction and Qiskit
"PauliEvolutionGate" for circuit construction.
qiskit_quimb pakage (https://github.com/qiskit-community/qiskit-quimb) is
utilized to convert Qiskit circuits into Quimb circuits.
Vector V_{\mu} calculation is not implemented.

Another implementation using solely Quimb works ~70% faster.

qiskit version = 1.1.1
quimb version = 1.8.4
qiskit-quimb version = 0.0.2
cotengra version = 0.6.2
"""

In [1]:
import numpy as np
import pickle
import time
from tqdm import tqdm
from typing import (
    List,
    Optional,
    Tuple,
    Union
)
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp, Pauli, Statevector
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator as QiskitEstimator

import quimb as qu
import quimb.tensor as qtn

from qiskit_quimb import quimb_circuit

import cotengra as ctg

In [2]:
class Quimb_avqite_contractions_estimates:
    def __init__(
        self,
        num_qubits: int,
        g: float,
        filename: str
    ):
        self._num_qubits = num_qubits
        self._g = g
        self._filename = filename

        self._H = self.set_tfim_H()

        #reading out the ansatz from a file
        (self._ansatz_adaptvqite,
         self._params_ansatz) = self.read_adaptvqite_ansatz(
                                "adaptvqite/adaptvqite/data/ansatz_inp.pkle"
                                )

        with open("incars/incar"+self._filename) as fp:
            incar_content = fp.read()

        if incar_content[-(3+self._num_qubits):-3] == "0"*self._num_qubits:
            self._init_circuit = QuantumCircuit(self._num_qubits)
        # applying bit-flip if initial state is |11...1>
        elif incar_content[-(3+self._num_qubits):-3] == "1"*self._num_qubits:
            self._init_circuit = QuantumCircuit(self._num_qubits)
            self._init_circuit.x([i for i in range(self._num_qubits)])
        else:
            raise ImportError(
                "Reference state is assumed to be either |00...0> or "
                "|11...1> here."
            )

        self._init_circuit = quimb_circuit(self._init_circuit)

        self.pauli_rotation_qc_decomposed_list = [
                    self.pauli_rotation_qc_decomposed(
                            theta=self._params_ansatz[i],
                            pauli_string=self._ansatz_adaptvqite[i]
                            )
                    for i in range(len(self._ansatz_adaptvqite))
                    ]

        self.pauli_rotation_dag_qc_decomposed_list = [
                    self.pauli_rotation_qc_decomposed(
                            theta=-self._params_ansatz[i],
                            pauli_string=self._ansatz_adaptvqite[i]
                            )
                    for i in range(len(self._ansatz_adaptvqite))
                    ]

    def read_adaptvqite_ansatz(
        self,
        filename: str
    ):
        """
        Reads the ansatz from a file resulting from adaptvqite calculation.

        Parameters
        ----------
        filename : str
            Name of a file containing the results of adaptvqite calculation.
            Has to be given in .pkle format.

        Returns
        -------
        ansatz_adaptvqite : List[str]
            List of Pauli strings entering the ansatz.
        params_adaptvqite : List[float64]
            Parameters (angles) of the ansatz.
        """
        if filename[-5:] != '.pkle':
            raise ImportError("Ansatz file should be given in .pkle format")

        with open(filename, 'rb') as inp:
            data_inp = pickle.load(inp)
            ansatz_adaptvqite = data_inp[0]
            params_adaptvqite = data_inp[1]
            # params_adaptvqite = list(np.random.random(20))

        return ansatz_adaptvqite, params_adaptvqite


    def set_tfim_H(
        self,
        BC: str = "open",
        J: float = 1.0
    ) -> "SparsePauliOp":
        """
        Generates TFIM Hamiltonian in Qiskit "SparsePauliOp" format.

        Parameters
        ----------
        BC : periodic
            Boundary conditions.
            Relevant options: "periodic", "open".
        J : float
            Interaction strength value.
        """
        if BC == "open":
            ZZ_string_array = [
                "I"*i+"ZZ"+"I"*(self._num_qubits-2-i)
                for i in range(self._num_qubits-1)
            ]
            J_array = [-J for i in range(self._num_qubits-1)]
        elif BC == "periodic":
            ZZ_string_array = [
                "I"*i+"ZZ"+"I"*(self._num_qubits-2-i)
                for i in range(self._num_qubits-1)
            ] + ["Z"+"I"*(self._num_qubits-2)+"Z"]
            J_array = [-J for i in range(self._num_qubits)]
        else:
            raise ValueError(
                "Not supported boundary conditions. "
                "Possible options: 'periodic', 'open'."
            )
        X_string_array = [
            "I"*i+"X"+"I"*(self._num_qubits-1-i)
            for i in range(self._num_qubits)
        ]
        g_array = [self._g for i in range(self._num_qubits)]

        H = SparsePauliOp(
            ZZ_string_array + X_string_array, coeffs = J_array + g_array
        )

        return H


    def pauli_rotation_gate(
        self,
        theta,
        pauli_string: str,
    ):
        """
        Generates a Pauli string rotation gate.

        Parameters
        ----------
        theta : float or "qiskit.circuit.parameter.Parameter"
            Pauli rotation angle.

        Returns
        -------
        gate : Qiskit instruction
        """
        operator = SparsePauliOp(pauli_string)
        gate = PauliEvolutionGate(operator, time = theta/2)
        return gate


    def pauli_rotation_qc_decomposed(
        self,
        theta,
        pauli_string: str,
    ):
        gate = self.pauli_rotation_gate(theta, pauli_string)
        qc = QuantumCircuit(self._num_qubits)
        qc.append(gate,range(self._num_qubits))
        qc = qc.decompose()
        return quimb_circuit(qc)


    def pauli_string_add_to_quimb_gates(
        self,
        gates,
        pauli_string
    ):
        for i,el in enumerate(pauli_string[::-1]):
            if el=='X':
                gates = gates + (qtn.circuit.Gate(
                                                label='X',
                                                params=[],
                                                qubits=(i,)
                                                ),
                                )
            if el=='Y':
                gates = gates + (qtn.circuit.Gate(
                                                label='Y',
                                                params=[],
                                                qubits=(i,)
                                                ),
                                )
            if el=='Z':
                gates = gates + (qtn.circuit.Gate(
                                                label='Z',
                                                params=[],
                                                qubits=(i,)
                                                ),
                                )
        return gates


    def circuit_1(
        self,
        mu: int,
        nu: int,
        A_mu: Union["Instruction", "Operator"],
        A_nu: Union["Instruction", "Operator"]
    ):
        if mu >= nu:
            raise ValueError("Here mu<nu is required.")
        if (mu > len(self._ansatz_adaptvqite) or
            nu > len(self._ansatz_adaptvqite) ):
            raise ValueError(
                "mu, nu has to be smaller"
                "than the number of operators in the ansatz"
            )

        circuit = self._init_circuit.copy()
        gates = ()

        t1=time.time()
        for i in range(mu):
            circuit.apply_gates(
                self.pauli_rotation_qc_decomposed_list[i].gates,
                contract=False
            )
        t2=time.time()
        circuit.apply_gates(
            self.pauli_string_add_to_quimb_gates(
                                                gates=gates,
                                                pauli_string=A_mu
                                                ),
                    contract=False )
        t3=time.time()
        for i in range(mu,nu):
             circuit.apply_gates(
                self.pauli_rotation_qc_decomposed_list[i].gates,
                contract=False
            )
        t4=time.time()
        circuit.apply_gates(
            self.pauli_string_add_to_quimb_gates(
                                                gates=gates,
                                                pauli_string=A_nu
                                                ),
                    contract=False )
        t5=time.time()
        for i in reversed(range(nu)):
            circuit.apply_gates(
                self.pauli_rotation_dag_qc_decomposed_list[i].gates,
                contract=False
                )
        t6=time.time()
        # circuit.apply_gates(gates)


        # print(t2-t1,t3-t2,t4-t3,t5-t4,t6-t5)
        return circuit


    def circuit_2(
        self,
        mu: int
    ):
        circuit = self._init_circuit.copy()
        gates = ()

        for i in range(mu):
            gates = gates + self.pauli_rotation_qc_decomposed_list[i].gates

        circuit.apply_gates(gates)

        return circuit


    def quimb_ampl_contr_est(
        self,
        circuit,
        opt = 'hyper'
    ):
        t3 = time.time()
        reh = circuit.amplitude_rehearse(
            '0'*self._num_qubits,
            optimize=opt
        )
        t4 = time.time()
        width, cost = (reh['tree'].contraction_width(),
                        reh['tree'].contraction_cost() )
        t5 = time.time()
        contraction = reh['tn'].contract(
            all,
            optimize=reh['tree'],
            output_inds=()
        )
        t6 = time.time()

        # print(t4-t3,t5-t4,t6-t5)
        return width, cost, contraction


    def quimb_local_exp_contr_est(
        self,
        circuit,
        operator,
        where,
        opt = 'hyper'
    ):
        reh = circuit.local_expectation_rehearse(
            operator,
            where,
            optimize=opt
        )

        width, cost = (reh['tree'].contraction_width(),
                        reh['tree'].contraction_cost())

        contraction = reh['tn'].contract(
            all,
            optimize=reh['tree'],
            output_inds=()
        )

        return width, cost, contraction


    def avqite_contr_est(
        self,
        contr_type: int,
        mu: Optional[int] = None,
        nu: Optional[int] = None,
    ):
        if contr_type == 1 and type(mu) != int and type(nu) != int:
            raise ValueError(
                "For contraction type 1,"
                " both mu and nu have to be int"
            )
        if contr_type == 2 and type(mu) != int and nu != None:
            raise ValueError(
                "For contraction type 2,"
                " mu has to be int and nu has to be None"
            )
        if contr_type == 3 and type(mu) != int and nu != None:
            raise ValueError(
                "For contraction type 3,"
                " mu has to be int and nu has to be None"
            )
        if contr_type == 4 and mu != None and nu != None:
            raise ValueError(
                "For contraction type 4,"
                " mu has to be None and nu has to be None"
            )
        if contr_type == 5 and mu != None and nu != None:
            raise ValueError(
                "For contraction type 5,"
                " mu has to be None and nu has to be None"
            )

        if mu != None and nu != None:
            if (mu > len(self._ansatz_adaptvqite) or
                nu > len(self._ansatz_adaptvqite) ):
                raise ValueError(
                    "mu, nu has to be smaller than the"
                    " number of operators in the ansatz"
                )

        if contr_type == 1:
            if mu<nu:
                t1=time.time()
                qc = self.circuit_1(
                    mu,
                    nu,
                    A_mu = self._ansatz_adaptvqite[mu],
                    A_nu = self._ansatz_adaptvqite[nu]
                )
                t2=time.time()
                width, cost, contraction = self.quimb_ampl_contr_est(
                                                circuit = qc
                                            )
                t3=time.time()
                # print(t2-t1,t3-t2)
            if mu==nu:
                width, cost, contraction = (1,0,1)
            contraction = np.real(contraction)/4

        if contr_type == 2:
#             width = []
#             cost = []
#             contraction_all = []

#             if mu < len(self._ansatz_adaptvqite):
#                 for H_string in self._H.paulis:
#                     qc = self.circuit_1(mu, len(self._ansatz_adaptvqite), A_mu = Pauli(self._ansatz_adaptvqite[mu]), A_nu = Pauli(H_string))
#                     width_one_string, cost_one_string, contraction_one_string = self.quimb_ampl_contr_est(circuit = qc)
#                     width.append(width_one_string)
#                     cost.append(cost_one_string)
#                     contraction_all.append(contraction_one_string)
#                 contraction = sum([self._H.coeffs[i]*contraction_all[i] for i in range(len(self._H.coeffs))])/2

#             if mu == len(self._ansatz_adaptvqite):
            raise NotImplementedError

        if contr_type == 3:
            qc = self.circuit_2(mu)
            where = [i
                    for i,p in enumerate(self._ansatz_adaptvqite[mu])
                    if p!= 'I']
            paulis = [p
                    for i,p in enumerate(self._ansatz_adaptvqite[mu])
                    if p!= 'I']

            operator = qu.pauli(paulis[0])
            for i in range(1,len(where)):
                operator = operator & qu.pauli(paulis[i])

            width, cost, contraction = self.quimb_local_exp_contr_est(
                                            circuit = qc,
                                            operator = operator,
                                            where = where
                                        )
            contraction = 1j*contraction/2

        return width, cost, contraction


In [4]:
test_obj = Quimb_avqite_contractions_estimates(num_qubits = 10, g=0.1, filename = "N10g0.1")

In [6]:
%%time
test_obj.avqite_contr_est(contr_type = 1, mu = 3, nu = 6)[-1]

CPU times: user 9.96 ms, sys: 708 μs, total: 10.7 ms
Wall time: 10.6 ms


0.0

In [7]:
%%time

M_munu = np.zeros((len(test_obj._ansatz_adaptvqite),len(test_obj._ansatz_adaptvqite)))

for nu in tqdm(range(len(test_obj._ansatz_adaptvqite))):
    for mu in range(nu+1):
        M_munu[mu,nu] = test_obj.avqite_contr_est(contr_type = 1, mu = mu, nu = nu)[-1] + test_obj.avqite_contr_est(contr_type = 3, mu = mu)[-1]*test_obj.avqite_contr_est(contr_type = 3, mu = nu)[-1]
        M_munu[nu,mu] = M_munu[mu,nu]

100%|██████████| 12/12 [00:01<00:00,  6.18it/s]

CPU times: user 1.83 s, sys: 12.3 ms, total: 1.84 s
Wall time: 1.94 s


In [8]:
with open("adaptvqite/adaptvqite/data/M_V.pkle", 'rb') as inp:
    data_inp = pickle.load(inp)
    M_adaptvqite = data_inp[0]

In [9]:
Mdiff = M_adaptvqite - M_munu
np.where((Mdiff>1e-14) == True)

(array([], dtype=int64), array([], dtype=int64))